# Warm-up examples

These following imports are necessary.

Worth mentioning are *stimator*, a library to support ODE-based modelling including parameter estimatiion from experimental data, and *ipywidgets*, on of the most popular widgets library for the jupyter notebook.

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as pl

from stimator import read_model
from ipywidgets import interact

## Closed system, reversible Michaelis-Menten enzyme

<span style="font-size:150%;">A ⇄ P</span>, catalyzed by an enzyme with kinetics

$$v = \frac{V \cdot (A - p /K_{eq})}{1 + A  / K_{mA} + P /K_{mP}}$$

In [ ]:
model = read_model("""
title Closed system, reversible Michaelis-Menten enzyme

# the reactions considered in the system
v1: A -> P, (V * (A - P / Keq)) / (1 + A/KmA + P/KmP)

# parameters
V = 1
Keq = 2

KmA = 2
KmP = 1

# initial state
init: A = 12, P = 0

""")

Function `solve()` generates a time course of concentrations (A and P).

Function `plot()` plots these time courses with sensible defaults

In [ ]:
model.solve(tf=40).plot(fig_size=(12,8))

Create and interactive cursor to change the initial value of A

In [ ]:
@interact(A=(6, 12, 0.5))
def change_A(A=12):
    model.init.A = A
    model.solve(tf=40).plot(fig_size=(12,8), yrange=(0,12))

### RESULT : in a closed system, concentrations tend to constant values with the equilibrium ratio

Now changing KmA

In [ ]:
model.init.A = 12
@interact(KmA=(1, 5, 0.1))
def change_KmA(KmA=2):
    model.parameters.KmA = KmA
    model.solve(tf=40).plot(fig_size=(12,8), yrange=(0,12))

## Open system (linear pathway), several reversible Michaelis-Menten enzymes

<span style="font-size:150%;">&ReverseEquilibrium; A ⇄ B ⇄ C ⇄ D &rarr;</span>

Not much difference between thye enzymesm except V of second enzyme is lower than the others

In [ ]:
model = read_model("""
title Open system, linear pathway

v1: A -> B, (V * (A - B / Keq)) / (1 + A/KmA + B/KmB), V = 1, Keq = 2, KmA = 2, KmB = 1
v2: B -> C, (V * (B - C / Keq)) / (1 + B/KmB + C/KmC), V = 0.5, Keq = 2, KmB = 2, KmC = 1
v3: C -> D, (V * (C - D / Keq)) / (1 + C/KmC + D/KmD), V = 1, Keq = 2, KmC = 2, KmD = 1

vAin  : -> A, kin, kin = 1
vAout : A ->, koutA * A, koutA = 1

vout  : D ->, kout * D, kout = 2
""")

model.solve(tf=20).plot(fig_size=(12,8))

# Save a SVG image to use somewhere else
pl.savefig('ss.svg')

Concentrations tend to constant values, again, but do they verify equilibrium constants (2 for all the reactions)??

In [ ]:
steady_state = model.solve(tf=20).last

print(steady_state, '\n')

A, B, C, D = steady_state['A'], steady_state['B'], steady_state['C'], steady_state['D']

print('B/A =', B/A)
print('C/B =', C/B)
print('D/C =', D/C)

### Nota at all: concentrations approach a **steady state**, where they are all constant but do not verify equilibrium constants

Now let's see what happens top the rates of reactions

In [ ]:
steady_state = model.solve(tf=20, outputs='>>').plot(fig_size=(12,8))

pl.savefig('flux.svg')

### At steady state, all reaction rates of the linear pathway have the same (net) value.

#### This value is called the *flux* of the pathway

In [ ]:
@interact(kin=(1, 5, 0.2))
def change_kin(kin=1):
    model.parameters.vAin.kin = kin
    model.solve(tf=20).plot(fig_size=(12,8), yrange=(0,4))

In [ ]:
@interact(kin=(1, 5, 0.2))
def change_kin(kin=1):
    model.parameters.vAin.kin = kin
    model.solve(tf=20, outputs='v1 v2 v3'.split()).plot(fig_size=(12,8), yrange=(0,1))

We can plot the effect of changing parameter *kin* on the steady-state concentrations.

In [ ]:
f,s = pl.subplots(1,1, figsize=(16,9))

values = np.linspace(1, 10, 20)
scan_kin = model.scan({'vAin.kin': values}, tf=30)

names = 'A', 'B', 'C', 'D'
colors = 'navy', 'darkgreen', 'firebrick', 'orange'
ss = {name: [tc.last[name] for tc in scan_kin] for name in names}

for name, color in zip(names, colors):
    pl.plot(values, ss[name], c=color, label=name, marker='o')

pl.xlabel('$k_{inA}$', fontsize=16)
pl.xlim(0,)
pl.ylim(0,10)
pl.grid()
legend = pl.legend()

### _kin_ seems to affect the steady-state concentrations differently.

Let's see the same effect but this time let's scale by the values arounf the steady state with _kin_ = 1

In [ ]:
f,s = pl.subplots(1,1, figsize=(16,9))

# get the steady state at kin = 1

model.parameters.vAin.kin = 1
steady_state = model.solve(tf=50).last

values = np.linspace(0.5, 1.5, 20)
scan_kin = model.scan({'vAin.kin': values}, tf=50)

names = 'A', 'B', 'C', 'D'
colors = 'navy', 'darkgreen', 'firebrick', 'orange'
ss = {name: [tc.last[name] for tc in scan_kin] for name in names}

scaled_values = values / 1
for name, color in zip(names, colors):
    pl.plot(values, ss[name] / steady_state[name], c=color, label=name, marker='o')

pl.xlabel('$k_{inA}$', fontsize=16)
pl.xlim(0.5, 1.5)
pl.ylim(0,2)
pl.grid()
legend = pl.legend()

#### Now the difference is not so dramatic.

## Sensitivities

Let's compute steady-state sensitivities to *kin*, by making 1% variations upper and lower.

### Steady-state concentration sensitivities to _kin_

In [ ]:
f,s = pl.subplots(1,1, figsize=(9,6))

# get the steady state at kin = 1

model.parameters.vAin.kin = 1
steady_state = model.solve(tf=50).last

kin = 1
values = np.array([kin*0.09, kin*1.01])
delta = (values[1] - values[0]) / kin
scan_kin = model.scan({'vAin.kin': values}, tf=50)

names = 'A', 'B', 'C', 'D'
colors = 'navy', 'darkgreen', 'firebrick', 'orange'
ss = {name: [tc.last[name] for tc in scan_kin] for name in names}

S = {}
for name in names:
    scaled = ss[name] / steady_state[name]
    S[name] = (scaled[1] - scaled[0]) / delta

pl.bar(S.keys(), S.values(), zorder=2, width=0.3, color=colors)
pl.ylabel('$S(X_{ss}, k_{in})$', fontsize=16)
pl.grid(zorder=-1)


Sensitivities are close to one, but there are differences

### Steady-state concentration sensitivities to $V_2$

In [ ]:
f,s = pl.subplots(1,1, figsize=(9,6))

# get the steady state at kin = 1

model.parameters.vAin.kin = 1
steady_state = model.solve(tf=50).last

V = 0.5
values = np.array([V*0.09, V*1.01])
delta = (values[1] - values[0]) / V
scan_v2V = model.scan({'v2.V': values}, tf=50)

names = 'A', 'B', 'C', 'D'
colors = 'navy', 'darkgreen', 'firebrick', 'orange'
ss = {name: [tc.last[name] for tc in scan_v2V] for name in names}

S = {}
for name in names:
    scaled = ss[name] / steady_state[name]
    S[name] = (scaled[1] - scaled[0]) / delta

pl.bar(S.keys(), S.values(), zorder=2, width=0.3, color=colors)
pl.ylabel('$S(X_{ss}, V_2)$', fontsize=16)
pl.grid(zorder=-1)


Now the results are completely different!